In [1]:
import pandas as pd
# import yfinance as yf

In [2]:
def fill_missing_returns(df):
    # Step 1: Calculate average returns for 1yr, 3yr, 5yr returns, ignoring missing values
    avg_1yr_return = df['1yr Annualized Return'].mean()
    avg_3yr_return = df['3yr Annualized Return'].mean()
    avg_5yr_return = df['5yr Annualized Return'].mean()

    # Step 2: Fill missing return values with average return - tracking error
    df['1yr Annualized Return'] = df.apply(
        lambda row: avg_1yr_return - row['Tracking Error'] if pd.isna(row['1yr Annualized Return']) else row['1yr Annualized Return'], axis=1)
    df['3yr Annualized Return'] = df.apply(
        lambda row: avg_3yr_return - row['Tracking Error'] if pd.isna(row['3yr Annualized Return']) else row['3yr Annualized Return'], axis=1)
    df['5yr Annualized Return'] = df.apply(
        lambda row: avg_5yr_return - row['Tracking Error'] if pd.isna(row['5yr Annualized Return']) else row['5yr Annualized Return'], axis=1)
    return df


In [43]:
def convert_to_float(df):
    df['Expense ratio'] = df['Expense ratio'].apply(lambda x: pd.to_numeric(x, errors='coerce') if isinstance(x, str) else x)
    df['Tracking Error'] = df['Tracking Error'].apply(lambda x: pd.to_numeric(x, errors='coerce') if isinstance(x, str) else x)
    df['AUM'] = df['AUM'].apply(lambda x: pd.to_numeric(x.replace(',', ''), errors='coerce') if isinstance(x, str) else x)
    df['1yr Annualized Return'] = df['1yr Annualized Return'].apply(lambda x: pd.to_numeric(x, errors='coerce') if isinstance(x, str) else x)
    df['3yr Annualized Return'] = df['3yr Annualized Return'].apply(lambda x: pd.to_numeric(x, errors='coerce') if isinstance(x, str) else x)
    df['5yr Annualized Return'] = df['5yr Annualized Return'].apply(lambda x: pd.to_numeric(x, errors='coerce') if isinstance(x, str) else x)
    return df

def normalize_data(df):
    # Normalizing the data
    df['Expense_ratio_normalized'] = (df['Expense ratio'].max() - df['Expense ratio']) / (df['Expense ratio'].max() - df['Expense ratio'].min())
    df['Tracking_Error_normalized'] = (df['Tracking Error'].max() - df['Tracking Error']) / (df['Tracking Error'].max() - df['Tracking Error'].min())
    df['AUM_normalized'] = (df['AUM'] - df['AUM'].min()) / (df['AUM'].max() - df['AUM'].min())

    # Normalizing the returns
    df['Return_normalized'] = (
        ((df['1yr Annualized Return'] - df['1yr Annualized Return'].min()) / (df['1yr Annualized Return'].max() - df['1yr Annualized Return'].min())) +
        ((df['3yr Annualized Return'] - df['3yr Annualized Return'].min()) / (df['3yr Annualized Return'].max() - df['3yr Annualized Return'].min())) +
        ((df['5yr Annualized Return'] - df['5yr Annualized Return'].min()) / (df['5yr Annualized Return'].max() - df['5yr Annualized Return'].min()))
    ) / 3  # Average of normalized returns
    
    return df

def calculate_total_score(df, weightages):
    # Step 4: Calculate the total score using weightages
    df['Total_Score'] = (
                        df['Expense_ratio_normalized'] * weightages['Expense ratio'] +
                        df['Tracking_Error_normalized'] * weightages['Tracking Error'] +
                        df['AUM_normalized'] * weightages['AUM'] +
                        df['Return_normalized'] * weightages['Return'])
    return df

def rank_funds(df):
    df['Rank'] = df['Total_Score'].rank(ascending=False)
    df = df.sort_values(by='Rank')
    return df


In [42]:
weightages = {'Expense ratio': 0.3, 'Tracking Error': 0.3, 'AUM': 0.15, 'Return': 0.25}

In [44]:
def process_fund_data(file_path, weightages):
    df = pd.read_csv(file_path)
    df = fill_missing_returns(df)
    df = convert_to_float(df)
    df = normalize_data(df)
    df = calculate_total_score(df, weightages)
    df = rank_funds(df)
    return df

In [45]:
display_columns = ['Fund','Expense ratio','Tracking Error','1yr Annualized Return', '2yr Annualized Return', '3yr Annualized Return', '5yr Annualized Return', 'Total_Score', 'Rank']
nifty50_df = process_fund_data('../data/nifty50.csv', weightages)
nifty50_df[display_columns].head(10)

,Fund,Expense ratio,Tracking Error,1yr Annualized Return,2yr Annualized Return,3yr Annualized Return,5yr Annualized Return,Total_Score,Rank
15,UTI Nifty 50 Index Fund,0.18,0.03,25.68,19.15,13.82,18.700000,0.731881,1.0
10,Navi Nifty 50 Index Fund,0.06,0.04,25.69,19.18,13.83,18.383571,0.712171,2.0
7,ICICI Prudential Nifty 50 Index Fund,0.17,0.04,25.61,19.10,13.78,18.690000,0.664465,3.0
5,HDFC Index Fund Nifty 50 Plan,0.20,0.05,25.63,19.12,13.78,18.610000,0.656267,4.0
2,Bandhan Nifty 50 Index Fund,0.10,0.08,25.68,19.18,13.91,18.880000,0.643310,5.0
12,SBI Nifty Index Fund,0.20,0.02,25.70,19.15,13.82,18.540000,0.621237,6.0
9,Motilal Oswal Nifty 50 Index Fund,0.15,0.04,25.76,19.19,13.86,18.383571,0.587741,7.0
16,Axis Nifty 100 Index Fund,0.21,0.08,30.98,20.32,14.74,18.343571,0.555023,8.0
11,Nippon India Index Nifty 50,0.20,0.05,25.61,19.09,13.78,18.630000,0.536007,9.0
3,DSP Nifty 50 Index Fund,0.18,0.07,25.69,19.13,13.79,18.560000,0.521066,10.0


In [46]:
nn50_df = process_fund_data('../data/niftynext50.csv', weightages)
nn50_df[display_columns].head(10)

,Fund,Expense ratio,Tracking Error,1yr Annualized Return,2yr Annualized Return,3yr Annualized Return,5yr Annualized Return,Total_Score,Rank
5,ICICI Prudential Nifty Next 50 Index Fund,0.31,0.09,59.82,30.00,20.720000,23.38,0.616634,1.0
10,UTI Nifty Next 50 Index Fund,0.37,0.06,59.82,29.99,20.720000,23.59,0.588875,2.0
9,SBI Nifty Next 50 Index Fund,0.33,0.05,59.87,30.03,20.770000,23.43,0.554102,3.0
2,DSP Nifty Next 50 Index Fund,0.28,0.12,59.84,30.14,20.860000,23.47,0.520460,4.0
6,Kotak Nifty Next 50 Index Fund,0.33,0.12,59.69,30.21,20.980000,23.36,0.417261,5.0
7,Motilal Oswal Nifty Next 50 Index Fund,0.35,0.09,59.74,30.09,20.760000,23.39,0.415922,6.0
3,HDFC NIFTY Next 50 Index Fund,0.30,0.12,59.44,29.76,20.632857,23.36,0.381905,7.0
0,Aditya Birla Sun Life Nifty Next 50 Index Fund,0.33,0.12,59.69,29.82,20.632857,23.36,0.358119,8.0
8,Navi Nifty Next 50 Index Fund,0.12,0.26,59.45,29.97,20.492857,23.22,0.321123,9.0
4,HSBC Nifty Next 50 Index Fund,0.35,0.09,59.49,29.77,20.460000,23.39,0.318108,10.0


In [47]:
midcap150_df = process_fund_data('../data/midcap150.csv', weightages)
midcap150_df[display_columns].head(10)

,Fund,Expense ratio,Tracking Error,1yr Annualized Return,2yr Annualized Return,3yr Annualized Return,5yr Annualized Return,Total_Score,Rank
1,Motilal Oswal Nifty Midcap 150,0.30,0.06,43.00,35.0,26.0000,31.00,0.787737,1.0
3,Nippon India Nifty Midcap 150,0.30,0.08,42.00,34.0,26.0000,30.92,0.634624,2.0
5,ICICI Prudential Nifty Midcap 150,0.30,0.08,42.00,34.0,27.7725,30.92,0.562343,3.0
2,HDFC Nifty Midcap 150,0.30,0.09,42.00,NaN,27.7625,30.91,0.518051,4.0
0,Navi Nifty Midcap 150 Index Fund,0.21,0.25,42.07,34.3,33.4100,30.75,0.474526,5.0
4,SBI Nifty Midcap 150,0.41,0.05,42.00,NaN,27.8025,30.95,0.468110,6.0
6,Aditya Birla Sun Life Nifty Midcap 150,0.44,0.31,43.00,35.0,26.0000,30.69,0.090465,7.0


In [48]:
smallcap250_df = process_fund_data('../data/smallcap250.csv', weightages)
smallcap250_df[display_columns].head(10)

,Fund,Expense ratio,Tracking Error,1yr Annualized Return,2yr Annualized Return,3yr Annualized Return,5yr Annualized Return,Total_Score,Rank
6,Motilal Oswal Nifty Smallcap 250 Index Fund,0.36,0.07,48.26,38.54,26.1100,32.56,0.629218,1.0
4,ICICI Prudential Nifty Smallcap 250 Index Fund,0.33,0.07,48.00,38.35,24.9000,32.49,0.590435,2.0
7,Nippon India Nifty Smallcap 250 Index Fund,0.35,0.22,47.82,38.31,25.9900,32.34,0.566623,3.0
8,SBI Nifty Smallcap 250 Index Fund,0.41,0.07,48.03,NaN,24.2025,32.49,0.562902,4.0
3,HDFC NIFTY Smallcap 250 Index Fund,0.30,0.11,47.85,NaN,24.1625,32.45,0.559139,5.0
9,Edelweiss Nifty Smallcap 250 Index Fund,0.14,0.26,48.70,NaN,24.0125,32.30,0.559105,6.0
5,Motilal Oswal Nifty Microcap 250 Index Fund,0.44,0.25,52.53,NaN,24.0225,32.31,0.451302,7.0
1,Axis Nifty Smallcap 50 Index Fund,0.28,0.35,55.02,42.04,23.9225,32.21,0.444355,8.0
2,Kotak Nifty Smallcap 50 Index Fund,0.38,0.43,55.10,NaN,23.8425,32.13,0.248228,9.0
0,Aditya Birla Sun Life Nifty Smallcap 50 Index ...,0.46,0.47,54.34,42.03,20.0900,32.09,0.087673,10.0
